In [1]:
from olsq import OLSQ
from olsq.device import qcdevice

from qibo import gates
from qibo.cskim_utils.gen_circuit import random_cx_circuit_qiskit
from qibo.models.circuit import Circuit

from qibo.cskim_utils.circuit_info import count_czs_qibo
from qibo.cskim_utils.connectivity import star_connectivity
from qibo.cskim_utils.connectivity import star_connectivity_cmap
from qibo.cskim_utils.gen_circuit import gen_transpiled_circuits_qibo
from qibo.transpiler.placer import ReverseTraversal
from qibo.transpiler.router import Sabre

from qiskit import transpile
from qibo.cskim_utils.circuit_info import count_czs_qiskit


# Compare the results of OLSQ and Qibo Router for a 5 qubit circuit

In [42]:
# Random CX circuit from Qiskit
random_circ_qiskit = random_cx_circuit_qiskit(5, 20)
print("Random circuit Qiskit:")
print(random_circ_qiskit.draw(fold=-1))

# Convert Qiskit circuit to Qibo circuit
random_circ_qibo = Circuit(5)
for g in random_circ_qiskit.data:
    q1 = g[1][0]._index
    q2 = g[1][1]._index
    random_circ_qibo.add(gates.CNOT(q1, q2))

print("Random circuit Qibo:")
print(random_circ_qibo.draw(line_wrap=1000))

Random circuit Qiskit:
               ┌───┐               ┌───┐                              ┌───┐     ┌───┐     
q_0: ──────────┤ X ├──■─────────■──┤ X ├──■───────────────────────────┤ X ├─────┤ X ├──■──
               └─┬─┘  │         │  └─┬─┘  │            ┌───┐          └─┬─┘     └─┬─┘  │  
q_1: ──■─────────┼────┼────■────┼────■────┼────────────┤ X ├────────────┼─────────┼────┼──
     ┌─┴─┐       │    │  ┌─┴─┐  │       ┌─┴─┐          └─┬─┘┌───┐┌───┐  │         │  ┌─┴─┐
q_2: ┤ X ├──■────■────┼──┤ X ├──┼───────┤ X ├────────────┼──┤ X ├┤ X ├──┼────■────┼──┤ X ├
     └───┘  │  ┌───┐┌─┴─┐└───┘┌─┴─┐     └───┘┌───┐┌───┐  │  └─┬─┘└─┬─┘  │  ┌─┴─┐  │  └───┘
q_3: ───────┼──┤ X ├┤ X ├─────┤ X ├──■────■──┤ X ├┤ X ├──■────┼────■────┼──┤ X ├──■───────
          ┌─┴─┐└─┬─┘└───┘     └───┘┌─┴─┐┌─┴─┐└─┬─┘└─┬─┘       │         │  └───┘          
q_4: ─────┤ X ├──■─────────────────┤ X ├┤ X ├──■────■─────────■─────────■─────────────────
          └───┘                    └───┘└───┘                      

## Qiskit Transpilation

- Additional optimizations exist in the Qiskit transpiler.

In [43]:
cmap = star_connectivity_cmap()

qc_basis = transpile(random_circ_qiskit, basis_gates=['cz', 'u'], optimization_level=3, coupling_map=cmap)

print("#czs after_transpiling", count_czs_qiskit(qc_basis))

# print(random_circ_qiskit.draw(fold=-1))
# print(qc_basis.draw(fold=-1))

#czs after_transpiling 26


## Qibo Transpilation

In [49]:
# Placer and Router
conn = star_connectivity()
router_sabre = Sabre(connectivity=conn)
placer = ReverseTraversal(connectivity=conn, routing_algorithm=router_sabre)

# Transpile
transpiled_circ = gen_transpiled_circuits_qibo(random_circ_qibo, conn, placer, router_sabre)
print("#czs after_transpiling", count_czs_qibo(transpiled_circ))

#czs after_transpiling 35


## OLSQ Routing

- OLSQ formulates the layout synthesis problem as a satisfiability modulo theories (SMT) optimization problem. 
- This problem is then passed to the Z3 SMT solver to obtain optimal solutions.



In [50]:
from qiskit.qasm2 import dumps

# Convert Qiskit circuit to QASM 2.0
qasm_str = dumps(random_circ_qiskit)

# initiate olsq with depth as objective, in normal mode
lsqc_solver = OLSQ("depth", "normal")

# directly construct a device from properties needed by olsq
# star connectivity
lsqc_solver.setdevice( qcdevice(name="dev", nqubits=5,
     connection=[(0, 2), (1, 2), (3, 2), (4, 2)], swap_duration=3) )

# load the qasm string to the solver
lsqc_solver.setprogram(qasm_str)

# solve the problem
_, result, _ = lsqc_solver.solve()

Trying maximal depth = 14...
Trying maximal depth = 18...
Trying maximal depth = 23...
Trying maximal depth = 29...
Trying maximal depth = 37...
Compilation time = 0:01:22.459291.
SWAP on physical edge (1,2) at time 15
SWAP on physical edge (3,2) at time 28
SWAP on physical edge (4,2) at time 5
SWAP on physical edge (4,2) at time 21
Gate 0: cx 1, 2 on qubits 1 and 2 at time 0
Gate 1: cx 2, 4 on qubits 2 and 0 at time 1
Gate 2: cx 4, 3 on qubits 0 and 2 at time 6
Gate 3: cx 2, 0 on qubits 2 and 3 at time 2
Gate 4: cx 0, 3 on qubits 3 and 2 at time 7
Gate 5: cx 1, 2 on qubits 2 and 4 at time 16
Gate 6: cx 0, 3 on qubits 3 and 2 at time 8
Gate 7: cx 3, 4 on qubits 2 and 0 at time 9
Gate 8: cx 1, 0 on qubits 2 and 3 at time 17
Gate 9: cx 3, 4 on qubits 2 and 0 at time 10
Gate 10: cx 4, 3 on qubits 0 and 2 at time 11
Gate 11: cx 4, 3 on qubits 0 and 2 at time 12
Gate 12: cx 3, 1 on qubits 1 and 2 at time 18
Gate 13: cx 0, 2 on qubits 3 and 2 at time 22
Gate 14: cx 4, 2 on qubits 0 and 2 at 